In [46]:
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold
from random import sample

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
n_samples = 5 #features to sample from each dataset
pixels = 40 #image size
perplexity = 25 #perplexity for the tsne

#Model
nruns = 2 #number of runs
folds = 10 #number of folds
features = 1 #number of features to predict
epochs = 180 #number of epochs
optimiser = 'adam' #model optimiser
loss = 'mse' #model loss

In [48]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [49]:
#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [ ]:
albums = []
for predictor_index in tqdm(range(len(omics))):
    
    #Get your predictor dataset
    predictors = list(omics)
    predictors.remove(omics[predictor_index])
    predictors = tuple(predictors)

    albums.append(butterfly.album.create_album(DF, predictors, pixels, perplexity))

 29%|██▊       | 2/7 [31:40<1:07:54, 814.96s/it]

In [ ]:
with open('albums.pkl', 'wb') as f:  
    pickle.dump(albums, f)

In [ ]:
with open('albums.pkl', 'rb') as f:
    albums = pickle.load(f)

In [ ]:
prediction_train = []
observed_train = []
prediction_test = []
observed_test = []

for predictor_index in tqdm(range(len(omics))):

    #Get your response dataset
    response = sample([col for col in DF if col.startswith(omics[predictor_index])], n_samples)
    response = np.append(response, 'patientID')
    response_df = DF[response]
    
    groups = response_df['patientID']
    
    y = response_df.values
    y = response_df.drop(['patientID'], axis = 1).values
    
    X = np.asarray(albums[predictor_index])
    
    prediction_train_CNN = []
    observed_train_CNN = []
    prediction_test_CNN = []
    observed_test_CNN = []

    for feat in tqdm(range(y.shape[1])):

        prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=nruns)
                               (delayed(butterfly.Models.CNN)
                                (X, y[:,feat], groups, pixels, features, folds, epochs, optimiser, loss, "CNN") 
                                for cv in range(nruns)))                

        prediction_train_CNN.append(prediction_train)
        observed_train_CNN.append(observed_train)
        prediction_test_CNN.append(prediction_test)
        observed_test_CNN.append(observed_test)
        
    prediction_train.append(prediction_train)
    observed_train.append(observed_train)
    prediction_test.append(prediction_test)
    observed_test.append(observed_test)

 20%|██        | 2/10 [01:21<05:06, 38.28s/it]/home/mxenoc/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 10/10 [06:43<00:00, 40.39s/it]
